# Calculate the percentage of anomaly for a specific basin
#### Jose Valles (jose.valles.leon@gmail.com)

### Importing Libraries

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('classic')
import numpy as np
import calendar

sns.set()

from IPython.display import HTML

In [73]:
# Define the basin level 2 code 
codcuenca_n2 = 61
# Define the variables
# model_variable = 'Escorrentia'

In [74]:
basin_level3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',usecols=lambda col: col.startswith(str(codcuenca_n2)))
basin_level2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',usecols=lambda col: col.startswith(str(codcuenca_n2)))

In [75]:
def importmodelvariable(codcuenca_n2,model_variable):
    df = pd.read_csv(f'../output_modelo/{model_variable}.csv',usecols=lambda col: col.startswith((str(codcuenca_n2),'-1')))
    df = df.rename(columns={'-1': 'year','-1.1':'month'})
    df['date'] = pd.to_datetime(dict(year=df['year'],month=df['month'],day=1))
    df = df.set_index('date')
    df['days_in_month'] = df.index.days_in_month
    return df

def convertRunoff2Discharge(df_runoff):
    df_runoff_selected = df_runoff.drop(['year','month','days_in_month'],axis=1)
    df_discharge = pd.DataFrame(df_runoff_selected.values*1000*basin_level3.values,columns=df_runoff_selected.columns)
    df_discharge['days_in_month'] = df_runoff['days_in_month'].values
    df_discharge = df_discharge.loc[:, df_discharge.columns != 'days_in_month'].divide(df_discharge["days_in_month"]*24*3600, axis="index")
    df_discharge['date'] = df_runoff.index.values
    df_discharge = df_discharge.set_index('date')
    df_discharge['year'] = df_runoff['year'].values
    df_discharge['month'] = df_runoff['month'].values
    return df_discharge

def defineHydroSOScategory(VARIABLE_MENSUAL,VARIABLE_AVERAGE,VARIABLE):
    # create empty columns in the dataframe
    VARIABLE_MENSUAL['mean'] = np.nan
    VARIABLE_MENSUAL['average_percentage'] = np.nan
    VARIABLE_MENSUAL['rank_average'] = np.nan
    VARIABLE_MENSUAL['non_missing'] = np.nan


    for i in range(len(VARIABLE_MENSUAL)):
        # Extract the current month 
        m = VARIABLE_MENSUAL.month[i]
        # Extract the current year
        y = VARIABLE_MENSUAL.year[i]
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'rank_average']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].rank()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'non_missing']  = VARIABLE_MENSUAL.query('month==@m')[VARIABLE].notnull().sum()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'mean'] = VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()
        VARIABLE_MENSUAL.loc[VARIABLE_MENSUAL.eval('month==@m & year==@y'),'average_percentage'] = (VARIABLE_MENSUAL[VARIABLE][i] - VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()) / VARIABLE_AVERAGE.query('month == @m')[VARIABLE].item()

    VARIABLE_MENSUAL['percentile'] = VARIABLE_MENSUAL['rank_average']/(VARIABLE_MENSUAL['non_missing']+1)
    criteria = [VARIABLE_MENSUAL['percentile'].between(0.87,1.00),
            VARIABLE_MENSUAL['percentile'].between(0.72,0.87),
            VARIABLE_MENSUAL['percentile'].between(0.28,0.72),
            VARIABLE_MENSUAL['percentile'].between(0.13,0.28),
            VARIABLE_MENSUAL['percentile'].between(0.00,0.13)]

    values = ['High flow','Above normal','Normal range','Below normal','Low flow']

    VARIABLE_MENSUAL['percentile_range'] = np.select(criteria,values,None)
    return VARIABLE_MENSUAL

In [76]:
hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETR','HumedadSuelo']

for hydro in hydrological_variable:
    if hydro == "Escorrentia_total":
        RUNOFF_total = importmodelvariable(codcuenca_n2, hydro)
    elif hydro == "Pmedias":
        PRECIP = importmodelvariable(codcuenca_n2, hydro)
    elif hydro == "ETR":
        ETR = importmodelvariable(codcuenca_n2, hydro)
    elif hydro == "HumedadSuelo":
        SM = importmodelvariable(codcuenca_n2, hydro)
    elif hydro == 'Escorrentia_sup':
        RUNOFF_sup = importmodelvariable(codcuenca_n2, hydro)
    elif hydro == 'Escorrentia_sub':
        RUNOFF_sub = importmodelvariable(codcuenca_n2, hydro)

In [ ]:
# # BORRAR

# hydrological_variable = ['Escorrentia_total','Escorrentia_sup','Escorrentia_sub','Pmedias','ETR','HumedadSuelo']

# for hydro in hydrological_variable:
#     locals()[hydro] = importmodelvariable(codcuenca_n2,hydro)

### Importar datos de modelos

Importar datos de precipitación, escorrentia, humedad suelo y Evapotranspiración (potencial y actual)

In [ ]:
# # BORRAR 

# # Escorrentia Total
# RUNOFF_total = pd.read_csv('../output_modelo/Escorrentia_total.csv')
# RUNOFF_total = RUNOFF_total.rename(columns={'-1': 'year','-1.1':'month'})
# RUNOFF_total['date'] = pd.to_datetime(dict(year=RUNOFF_total['year'],month=RUNOFF_total['month'],day=1))
# RUNOFF_total = RUNOFF_total.set_index('date')
# RUNOFF_total['days_in_month'] = RUNOFF_total.index.days_in_month

# # Escorrentia superficial
# RUNOFF_sup = pd.read_csv('../output_modelo/Escorrentia_sup.csv')
# RUNOFF_sup = RUNOFF_sup.rename(columns={'-1': 'year','-1.1':'month'})
# RUNOFF_sup['date'] = pd.to_datetime(dict(year=RUNOFF_sup['year'],month=RUNOFF_sup['month'],day=1))
# RUNOFF_sup = RUNOFF_sup.set_index('date')
# RUNOFF_sup['days_in_month'] = RUNOFF_sup.index.days_in_month

# # Precipitación
# PRECIP = pd.read_csv('../output_modelo/Pmedias.csv')
# PRECIP = PRECIP.rename(columns={'-1': 'year','-1.1':'month'})
# PRECIP['date'] = pd.to_datetime(dict(year=PRECIP['year'],month=PRECIP['month'],day=1))

# # Evapotranspiración Real
# ETR = pd.read_csv('../output_modelo/ETR.csv')
# ETR = ETR.rename(columns={'-1': 'year','-1.1':'month'})
# ETR['date'] = pd.to_datetime(dict(year=ETR['year'],month=ETR['month'],day=1))

# # Humedad Suelo 
# SM = pd.read_csv('../output_modelo/HumedadSuelo.csv')
# SM = SM.rename(columns={'-1': 'year','-1.1':'month'})
# SM['date'] = pd.to_datetime(dict(year=SM['year'],month=SM['month'],day=1))


Importar codigos de cuenca nivel 2 y 3

In [77]:
BASIN_LEVEL3 = pd.read_csv(f'../output_modelo/cuenca_nivel3.csv',index_col="Codigo")
BASIN_LEVEL2 = pd.read_csv(f'../output_modelo/cuenca_nivel2.csv',index_col="Codigo")

In [81]:
HTML(RUNOFF_total.tail(6).to_html(index=False))

year,month,610,611,612,613,615,616,618,days_in_month
2022,12,3.4867,1.22200,0.224290,0.000797,0.351440,0.175640,0.105390,31
2023,1,1.2956,0.45406,0.083342,0.000296,0.130590,0.065264,0.039159,31
2023,2,0.4814,0.16872,0.030968,0.000110,0.048524,0.024250,0.014551,28
2023,3,12.3830,4.74530,2.294000,1.066400,0.077438,0.125920,0.005407,31
2023,4,7.2558,3.58160,1.949300,0.974470,0.064739,0.117100,0.002009,30
2023,5,2.6961,1.33080,0.724320,0.362090,0.024056,0.043511,0.000746,31


### Convertir Escorrentia (mm/month) a Caudal (m3/s)

In [82]:
DISCHARGE = convertRunoff2Discharge(RUNOFF_total)

In [ ]:
# # Convertir datos de escorrentia a caudal
# RUNOFF_SELECTION = RUNOFF_total.drop(['year','month','days_in_month'],axis=1)
# DISCHARGE = pd.DataFrame(RUNOFF_SELECTION.values*1000*BASIN_LEVEL3.values,columns=RUNOFF_SELECTION.columns)
# DISCHARGE['days_in_month'] = RUNOFF_sup['days_in_month'].values
# DISCHARGE = DISCHARGE.loc[:, DISCHARGE.columns != 'days_in_month'].divide(DISCHARGE["days_in_month"]*24*3600, axis="index")
# # Definir columns de date y año
# DISCHARGE['date'] = RUNOFF_total.index.values
# DISCHARGE = DISCHARGE.set_index('date')
# DISCHARGE['year'] = RUNOFF_total['year'].values
# DISCHARGE['month'] = RUNOFF_total['month'].values

In [83]:
HTML(DISCHARGE.tail(6).to_html(index=True))

,610,611,612,613,615,616,618,year,month
date,,,,,,,,,
2022-12-01,0.334559,0.212609,0.045052,0.000118,0.017976,0.045641,0.003148,2022,12
2023-01-01,0.124316,0.078999,0.016741,0.000044,0.006680,0.016959,0.001170,2023,1
2023-02-01,0.051141,0.032500,0.006887,0.000018,0.002748,0.006977,0.000481,2023,2
2023-03-01,1.188184,0.825608,0.460787,0.158065,0.003961,0.032721,0.000161,2023,3
2023-04-01,0.719422,0.643914,0.404600,0.149253,0.003422,0.031444,0.000062,2023,4
2023-05-01,0.258698,0.231539,0.145491,0.053670,0.001230,0.011307,0.000022,2023,5


### Pleasee select the runoff type for the analysis (RUNOFF_total, RUNOFF_sub, RUNOFF_sup)

In [84]:
# Select the runoff type 
RUNOFF = RUNOFF_total

### Select reference period from 'year_start' to 'year_end'

Select variable of interest

In [85]:
year_start = 1981
year_end = 2010
# Caudal
SELECTED_REF_DISCHARGE= DISCHARGE[(DISCHARGE['year'] >= year_start) & (DISCHARGE['year'] <= year_end)]
# Escorrentia
SELECTED_REF_RUNOFF = RUNOFF[(RUNOFF['year'] >= year_start) & (RUNOFF['year'] <= year_end)]
# Precip
SELECTED_REF_PRECIP = PRECIP[(PRECIP['year'] >= year_start) & (PRECIP['year'] <= year_end)]

#### Monthly analysis of the selected basin

Based on the codcuenca_n2, here presents the monthly analysis of the runoff variable

In [86]:
RUNOFF['avg_runoff'] = RUNOFF.iloc[:,2:-1].mean(axis=1)

In [87]:
sum_avg_runoff_year = RUNOFF.groupby(['year'])['avg_runoff'].sum()
sum_avg_runoff_year = sum_avg_runoff_year.to_frame()
sum_avg_runoff_month = RUNOFF.groupby(['year','month'])['avg_runoff'].sum()
sum_avg_runoff_month = sum_avg_runoff_month.to_frame()

In [89]:
HTML(sum_avg_runoff_month.tail(12).to_html())

In [90]:
# sum_avg_runoff_month.to_clipboard()

### Aggregate the basin Leve 3 discharge values to Basin Level 2 discharge

In [98]:
# Buscar columnas que empiezan con el codigo de cuenca nivel 2
filter_col = [col for col in DISCHARGE if col.startswith(str(codcuenca_n2))]
DISCHARGE_SELECTED = DISCHARGE[filter_col]
# Discharge 
DISCHARGE_N2 = pd.DataFrame()
DISCHARGE_N2['year'] = DISCHARGE['year']
DISCHARGE_N2['month'] = DISCHARGE['month']
DISCHARGE_N2['discharge'] = DISCHARGE_SELECTED.sum(axis=1)

DISCHARGE_N2['date'] = pd.to_datetime(dict(year=DISCHARGE_N2['year'],month=DISCHARGE_N2['month'],day=1))
DISCHARGE_N2 = DISCHARGE_N2.set_index('date')

In [100]:
HTML(DISCHARGE_N2.tail(12).to_html())

,year,month,discharge
date,,,
2022-06-01,2022,6,0.357227
2022-07-01,2022,7,14.049967
2022-08-01,2022,8,8.576951
2022-09-01,2022,9,4.492150
2022-10-01,2022,10,2.667636
2022-11-01,2022,11,1.609445
2022-12-01,2022,12,0.659103
2023-01-01,2023,1,0.244909
2023-02-01,2023,2,0.100751
